In [1]:
from src.project import Project

proj = Project(
    "/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh",
    "10_09_01MJPG-0000",
)

In [2]:
bg, calibration, feat, params, track = proj.read_project_data()

In [3]:
import pandas as pd
import numpy as np


def convert_data_to_df(data, col_names):
    n_flies, n_frames, n_flags = data.shape
    reshaped_data = data.reshape(-1, n_flags)

    df = pd.DataFrame(reshaped_data, columns=col_names)
    df["id"] = np.repeat(np.arange(n_flies), n_frames)

    return df

In [4]:
track_data = track["trk"]["data"]
feature_data = feat["feat"]["data"]

track_names = [i.replace(" ", "_") for i in track["trk"]["names"]]
feature_names = feat["feat"]["names"]

In [5]:
track_df = convert_data_to_df(track_data, track_names)
feature_df = convert_data_to_df(feature_data, feature_names)

In [6]:
# Concatenate along axis=1 (horizontally)
track_feature = pd.concat([feature_df, track_df], axis=1)

In [7]:
from src.project_dlc import ProjectDLC
from src.video import VideoInfo


complete_data_path = "/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh/dlc_output/20240115-1015-fly1-melWT_3do_sh_melWT_3do_ghDLC_dlcrnetms5_38mm-dyadFeb23shuffle1_50000_el.h5"

video_path = "/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh/10_09_01MJPG-0000.avi"
video = VideoInfo(video_path)
full_data = ProjectDLC.load_data(complete_data_path)

In [8]:
df_fly1 = ProjectDLC.process_single_fly(full_data, "fly1", video)
df_fly2 = ProjectDLC.process_single_fly(full_data, "fly2", video)

/Users/isabellechen/flytracker_analysis/src/dlc/measurement.py:57: RuntimeWarning: Mean of empty slice
  x_center, y_center = np.nanmean(x_coords, axis=1), np.nanmean(y_coords, axis=1)
/Users/isabellechen/flytracker_analysis/src/dlc/measurement.py:57: RuntimeWarning: Mean of empty slice
  x_center, y_center = np.nanmean(x_coords, axis=1), np.nanmean(y_coords, axis=1)


In [9]:
from src.dlc.relative_measurement import RelativeMeasurement

df_fly1, df_fly2 = RelativeMeasurement.get_interfly_params(
    df_fly1, df_fly2, "fly1", "fly2"
)

In [10]:
pd.set_option("display.max_columns", None)
track_feature

,vel,ang_vel,min_wing_ang,max_wing_ang,mean_wing_length,axis_ratio,fg_body_ratio,contrast,dist_to_wall,dist_to_other,angle_between,facing_angle,leg_dist,id,pos_x,pos_y,ori,major_axis_len,minor_axis_len,body_area,fg_area,img_contrast,min_fg_dist,wing_l_x,wing_l_y,wing_r_x,wing_r_y,wing_l_ang,wing_l_len,wing_r_ang,wing_r_len,leg_1_x,leg_1_y,leg_2_x,leg_2_y,leg_3_x,leg_3_y,leg_4_x,leg_4_y,leg_5_x,leg_5_y,leg_6_x,leg_6_y,leg_1_ang,leg_2_ang,leg_3_ang,leg_4_ang,leg_5_ang,leg_6_ang,id
0,27.760335,0.059031,0.070184,0.113363,1.904930,3.241620,3.105856,0.265803,2.909003,25.870275,2.526759,1.629047,23.510209,0,542.728604,167.671171,2.680097,62.571729,19.302612,888.0,2758.0,0.265803,661.777161,588.0,197.0,592.0,188.0,-0.113363,53.941445,0.070184,53.300392,564.0,205.0,558.0,137.0,515.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.591355,1.570320,2.736041,NaN,NaN,NaN,0
1,27.760335,0.287484,0.060691,0.102101,1.902980,3.322472,3.112842,0.269980,2.683556,25.764119,2.609928,1.652736,23.357690,0,530.068966,162.742529,2.681081,62.768310,18.870768,870.0,2725.0,0.270478,655.746887,576.0,191.0,579.0,184.0,-0.091021,53.927216,0.050674,53.349098,508.0,185.0,563.0,205.0,560.0,149.0,557.0,136.0,499.0,128.0,NaN,NaN,-1.891430,-0.448297,0.890940,1.242397,2.760944,NaN,0
2,22.893499,0.987162,0.072418,0.099556,1.895876,3.391456,3.173063,0.268889,2.517328,25.661251,2.681122,1.670508,23.300614,0,519.321143,159.102857,2.664867,63.827102,18.795738,875.0,2696.0,0.273161,656.665039,564.0,189.0,568.0,180.0,-0.112999,53.759087,0.071230,52.974727,478.0,148.0,508.0,184.0,560.0,149.0,531.0,137.0,499.0,129.0,NaN,NaN,-2.927364,-1.520840,0.720157,1.561423,2.641311,NaN,0
3,15.135936,1.529196,0.080854,0.093614,1.895622,3.361646,3.224714,0.268279,2.412099,25.550027,2.734242,1.667220,23.290097,0,512.599754,158.091133,2.697266,62.541858,18.138873,812.0,2759.0,0.258755,654.432556,559.0,185.0,562.0,176.0,-0.081204,53.638326,0.096537,52.546283,465.0,148.0,507.0,186.0,560.0,148.0,516.0,125.0,497.0,128.0,NaN,NaN,-2.906173,-1.324485,0.654087,1.912728,2.493395,NaN,0
4,6.958983,1.393339,0.072333,0.095410,1.903336,3.249895,3.123369,0.276265,2.389758,25.459197,2.779556,1.653372,23.307570,0,508.990033,157.931340,2.718201,62.548479,19.826057,903.0,2729.0,0.282446,656.795227,556.0,185.0,559.0,177.0,-0.099049,54.246192,0.059112,53.522057,468.0,148.0,506.0,183.0,523.0,189.0,560.0,148.0,516.0,124.0,497.0,128.0,-2.955907,-1.266117,-0.723773,0.615680,1.790462,2.375198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215995,16.330244,0.594309,0.020885,0.063930,2.200976,3.192546,2.480808,0.166200,5.350743,3.954129,0.145099,2.996135,0.387261,1,326.926081,890.956764,-0.874424,78.699979,24.591345,1434.0,3577.0,0.166228,11.180340,291.0,839.0,287.0,845.0,-0.091400,63.167940,0.018921,60.877879,350.0,860.0,336.0,860.0,274.0,881.0,297.0,929.0,345.0,946.0,NaN,NaN,-1.336897,-0.981502,0.688471,1.778683,2.762487,NaN,1
215996,16.729621,0.968933,0.020889,0.051766,2.192903,3.237779,2.493921,0.167686,5.325563,3.998637,0.179330,2.983556,0.596862,1,332.438472,896.395332,-0.900471,78.595022,24.258804,1414.0,3525.0,0.167931,17.000000,295.0,846.0,294.0,850.0,-0.031387,62.780003,0.021553,60.249838,359.0,887.0,297.0,893.0,298.0,928.0,350.0,956.0,NaN,NaN,NaN,NaN,-1.901137,0.804953,1.642987,2.757793,NaN,NaN,1
215997,18.949663,1.013950,0.020647,0.048799,2.201918,3.258233,2.505767,0.167309,5.305263,4.060245,0.212261,2.978088,0.761529,1,338.080565,902.954770,-0.910909,78.919620,24.126381,1415.0,3531.0,0.168656,22.022715,302.0,851.0,300.0,856.0,-0.052889,63.254291,0.021528,60.455603,378.0,890.0,305.0,908.0,299.0,928.0,350.0,956.0,NaN,NaN,NaN,NaN,-1.916883,1.062256,1.480833,2.702737,NaN,NaN,1
215998,19.587339,0.867803,0.022211,0.056043,2.214864,3.249690,2.528554,0.165846,5.254227,4.092901,0.222704,2.982379,0.753794,1,344.838483,911.037921,-0.931582,79.063065,24.320418,1424.0,3616.0,0

In [11]:
from collections import Counter


def filter_unique(lst):
    """
    Filters out unique values from a list, keeping only elements that appear more than once.

    Args:
        lst (list): Input list of values.

    Returns:
        list: List of values that appear more than once.
    """
    counts = Counter(lst)  # Count occurrences of each value
    return [item for item in lst if counts[item] == 1]


a = filter_unique(track_feature["dist_to_other"].to_list())

In [12]:
len(track_feature["dist_to_other"].to_list())

216000

In [13]:
len(a)

0

In [14]:
df_fly1

,ori,centroid_x,centroid_y,lin_speed,left_wing_angle,right_wing_angle,inter_wing_dist,body_length,time,dist_to_other,fly1_facing_angle,fly1_ang_between
0,1.092144,246.37900,815.36080,0.000000,-0.020222,0.005581,2.218317,65.573367,0.000000,714.948660,-2.211888,-2.580888
1,0.851593,245.93060,814.88660,7.831581,-0.136659,-0.101541,2.331655,59.682535,0.016667,710.386278,-1.990429,-2.673250
2,0.788169,247.05060,813.40300,22.306670,-0.123074,-0.083358,2.560145,62.533228,0.033334,707.847641,-1.944075,-2.746615
3,0.802668,251.29400,813.95660,51.352312,-0.038331,0.075875,7.550308,73.091092,0.050000,707.906575,-1.971763,-2.707187
4,0.837670,251.84300,814.68300,10.926314,0.047060,0.109841,4.336923,70.022846,0.066667,706.313020,-2.008824,-2.754267
...,...,...,...,...,...,...,...,...,...,...,...,...
107995,0.925545,311.39340,871.25920,74.581068,0.114412,0.105583,1.218072,71.662910,1799.933333,108.776719,3.014401,0.250883
107996,0.873943,311.31440,857.64200,163.409150,-0.072448,-0.210277,97.340948,73.190141,1799.950000,89.171222,2.957871,0.138462
107997,0.876846,310.72700,846.58460,132.875894,-0.260037,-0.223223,6.353279,73.784476,1799.966666,84.661336,2.687834,0.193211
107998,0.868896,311.86020,878.62460,384.720401,0.168901,0.004434,110.528832,74.033207,1799.983333,90.303015,3.137396,0.230418
